In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 2.9MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [60.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

In [13]:
import requests
import os
from bs4 import BeautifulSoup
from time import sleep
from urllib.request import urlopen
import json
import time
import pandas as pd
import io

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementNotVisibleException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('lang=ko_KR')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [14]:
#웹드라이버 로딩
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.implicitly_wait(3)

In [15]:
#크롤링 함수 정의+실행 부분입니다. 셀 실행 3시간 이상 걸립니다.

def main():
    global driver, load_wb, review_num, searchList,place_name
    
    searchList = []
    driver.implicitly_wait(4)  # 렌더링 될때까지 기다린다 4초
    driver.get('https://map.kakao.com/?')

    ###################이부분 바꿔주시면 됩니다!!!!!!!!!!!(1)###################

    place_infos = ['성수 식당'] #강남 식당:지영 #성수 식당:다현 #건대 식당:서연 #잠실 식당:지현 #종로 식당:형구 #신림 식당:수용

    ######################################################################
    for i, place in enumerate(place_infos):
        # delay
        if i % 4 == 0 and i != 0:
            sleep(5)
        print("#####", i)
        search(place)
        
    driver.quit()   
    # print(searchList)    
    print("finish")
    

def search(place):
    global driver, place_name

    search_area = driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]')  # 검색 창
    
    search_area.send_keys(place)  # 검색어 입력
    driver.find_element_by_xpath(
        '//*[@id="search.keyword.query"]').send_keys(Keys.ENTER)  # Enter로 검색
    sleep(3)

    # 검색된 정보가 있는 경우에만 탐색
    # 1번 페이지 place list 읽기
    html = driver.page_source

    soup = BeautifulSoup(html, 'html.parser')
    place_lists = soup.select('.placelist > .PlaceItem')  # 검색된 장소 목록


    ###################이부분 바꿔주시면 됩니다!!!!!!!!!!!(2)###################

    # 검색된 첫 페이지 장소 목록 크롤링하기
    # 더보기 눌렀을 때 경우에 따라 아래 주석처리를 바꿔줘야 합니다. "연세대 식당""건대 식당"->주석처리. "강남 식당""잠실 식당""성수 식당""종로 식당""신림 식당"->주석 풀어주기.
    crawling(place, place_lists)

    ######################################################################

    search_area.clear()

    # 우선 더보기 클릭해서 2페이지
    try:
        driver.find_element_by_xpath(
            '//*[@id="info.search.place.more"]').send_keys(Keys.ENTER)
        sleep(1)
        count = 0

        while True:
          if count == 0:
            # 2~ 5페이지 읽기
            ##################이부분 바꿔주시면 됩니다!!!!!!!!!!!(3)###################
            for i in range(2, 6): #더보기 눌렀을 때 경우에 따라 (1,6)일지 (2,6)일지 바꿔줘야합니다. "연세대 식당""건대 식당":(1,6) "강남 식당""잠실 식당""성수 식당""종로 식당""신림 식당":(2,6).
            ######################################################################
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem')  # 장소 목록 list

                # print(place_lists)

                crawling(place, place_lists)
            
            xPath = '//*[@id="info.search.page.next"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            count+=1
            sleep(1)
          elif count != 0:
            # 2~ 5페이지 읽기
            for i in range(1, 6):
                # 페이지 넘기기
                xPath = '//*[@id="info.search.page.no' + str(i) + '"]'
                driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
                sleep(1)

                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                place_lists = soup.select('.placelist > .PlaceItem')  # 장소 목록 list

                # print(place_lists)

                crawling(place, place_lists)
            
            xPath = '//*[@id="info.search.page.next"]'
            driver.find_element_by_xpath(xPath).send_keys(Keys.ENTER)
            sleep(1)

    except ElementNotInteractableException:
        print('not found')
    finally:
        search_area.clear()

def crawling(place, place_lists):
    """
    페이지 목록을 받아서 크롤링 하는 함수
    :param place: 리뷰 정보 찾을 장소이름
    """
    while_flag = False
    for i, place in enumerate(place_lists):
        # 광고에 따라서 index 조정해야함
        if i >= 3:
            i += 1

        place_name = place.select('.head_item > .tit_name > .link_name')[0].text  # place name
        place_address = place.select('.info_item > .addr > p')[0].text  # place address

        detail_page_xpath = '//*[@id="info.search.place.list"]/li[' + str(i + 1) + ']/div[5]/div[4]/a[1]'
        try:
          driver.find_element_by_xpath(detail_page_xpath).send_keys(Keys.ENTER)
        except NoSuchElementException:
          continue
        driver.switch_to.window(driver.window_handles[-1])  # 상세정보 탭으로 변환
        sleep(1)

        print('####', place_name)

        # 첫 페이지
        extract_review(place_name)

        # 2-5 페이지
        idx = 3
        try:
            page_num = len(driver.find_elements_by_class_name('link_page')) # 페이지 수 찾기
            for i in range(page_num-1):
                # css selector를 이용해 페이지 버튼 누르기
                driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                sleep(1)
                extract_review(place_name)
                idx += 1
            driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 5페이지가 넘는 경우 다음 버튼 누르기
            sleep(1)
            extract_review(place_name) # 리뷰 추출
        except (NoSuchElementException, ElementNotInteractableException):
            print("no review in crawling")

        # 그 이후 페이지
        while True:
            idx = 4
            try:
                page_num = len(driver.find_elements_by_class_name('link_page'))
                for i in range(page_num-1):
                    driver.find_element_by_css_selector('#mArticle > div.cont_evaluation > div.evaluation_review > div > a:nth-child(' + str(idx) +')').send_keys(Keys.ENTER)
                    sleep(1)
                    extract_review(place_name)
                    idx += 1
                driver.find_element_by_link_text('다음').send_keys(Keys.ENTER) # 10페이지 이상으로 넘어가기 위한 다음 버튼 클릭
                sleep(1)
                extract_review(place_name) # 리뷰 추출
            except (NoSuchElementException, ElementNotInteractableException):
                print("no review in crawling")
                break

        driver.close()
        driver.switch_to.window(driver.window_handles[0])  # 검색 탭으로 전환

def extract_review(place_name):
    global driver

    ret = True

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # 첫 페이지 리뷰 목록 찾기
    review_lists = soup.select('.list_evaluation > li')   
    
    # 리뷰가 있는 경우
    if len(review_lists) != 0:
        
        for i, review in enumerate(review_lists):
            
            comment = review.select('.txt_comment > span') # 리뷰
            rating = review.select('.grade_star > em') # 별점
            val = ''
            if len(comment) != 0:
                innersearchList = []
                if len(rating) != 0:
                    val = comment[0].text + '/' + rating[0].text.replace('점', '')
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(rating[0].text)
                else:
                    val = comment[0].text + '/0'
                    innersearchList.append(place_name)    
                    innersearchList.append(comment[0].text)
                    innersearchList.append(0)
                print(val)
                
                searchList.append(innersearchList)
    else:
        print('no review in extract')
        ret = False
    return ret
    
if __name__ == "__main__":
    main()

##### 0
#### 블루보틀 성수
no review in extract
#아메리카노#굿/4
..?/2
직원서비스는 최고인데, 오래 머무를 곳은 아니며 본점이지만 큰 매력은 못 느끼겠다. 굿즈는 가격 사악하지만 소장욕구 넘침/3
귀리우유로 만든 라떼 진짜 먹어본 것 중 최고였어요. 왜 블루보틀 하는지 알겠어요. 자리가 부족해서 아쉬웠지만 전반적인 경험은 좋았어요. /5
굳/5
그냥 그래요/2
최고예요/5
커피~~너무 맛있어용 맛있는커피 마시러가기 좋은곳 ! 작업공간은 콘센트도 와이파이도 없어서 불가능 ../5
보통이에요/3
생각보다 괜찮네요/4
* 아 취한드아 놀라 마티니 마시따 가격보고 두번 놀라/5
스벅보단 블루보틀이 좋아서 매장 더 늘어났으면. 사람 넘 몰린다../3
무난합니다. /3
아메리카노보다는 카페라테를 추천/4
스벅 갈게/2
최고예요/5
보통이에요/3
커피맛 산도있고 다양한맛이 나요  묘해요 아이스 드립 맛있었어요 가까우면 애정하는곳이 됬을듯/3
기대없이 갔는데 뉴올리언스 맛났어요.  자리는 불편했지만 가격만 아니면 괜춘할듯. /3
인스타용 맛은 별로/3
괜찮아요. 맛있어요./4
직원분들 너무너무너무 친절해요 서비스 교육은 진짜 최고이신듯 합니다/4
맛만 따진다면./5
보통이에요/3
정신없고 음료 맛도 없음./2
잘 모르겠음 세련된 분위기는 좋음/2
조금 아쉬워요/2
메뉴는 잘 골라야할듯 저는 추천받아 맛있게 마셨습니당/4
미국 블루보틀이랑 맛 비슷하고 맛있어요.  건축물이 감각적이고 멋있습니다!/5
주위 다른 카페들에 비해 가격이나 맛이나 그닥.../2
보통이에요/3
친구따라 가보긴했는데 그냥저냥 평범했어요 파란 물병 모양의 로고가 시그니처인듯 사람들이 많이 찍어가더라구요/3
지나가다가 가봤는데 괜찮음..ㅎ/4
한 번. 가볼만함 두번은 안감,,7000원대 라떼 수준은 아닌듯/3
맛 생각보다 기대이하. 사람 많음/3
전 커피맛은 만족이요. 시그니쳐 라테, 아메리카노 둘다 괜찮더라구요. 또 생각나는 맛이긴 합니다. 매장에 자리도 그정도면 괜찮구요

ProtocolError: ignored

In [5]:
df = pd.DataFrame(searchList)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26725 entries, 0 to 26724
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       26725 non-null  object
 1   1       26725 non-null  object
 2   2       26725 non-null  object
dtypes: object(3)
memory usage: 626.5+ KB


In [7]:
df.head(10)

,0,1,2
0,카페노티드 청담,알바생 응대 최악 돈주고 불친절을 경험하네요,1점
1,카페노티드 청담,정말정말 마케팅을 잘한예.. 맛은있지만 또기다리라면 못기다림 안먹고말지 여기말고도 ...,2점
2,카페노티드 청담,먀...,3점
3,카페노티드 청담,최고예요,5점
4,카페노티드 청담,케이크 포장은 줄이 길지 않아서 오래 안 기다렸구 케이크 크기가 딱 적당해서 너무 ...,4점
5,카페노티드 청담,좋아요,4점
6,카페노티드 청담,보통이에요,3점
7,카페노티드 청담,별루 현명하지 못하게 운영하는듯,1점
8,카페노티드 청담,노티드 최고!!!,5점
9,카페노티드 청담,"기다리기 너무 힘든데 맛은 겁나 맛잇음,,,,,,",4점


In [8]:
df.rename(columns = {0 : 'total_name'}, inplace = True)
df.rename(columns = {1 : 'comment'}, inplace = True)
df.rename(columns = {2 : 'rate'}, inplace = True)
final_df=df.assign(source='카카오')

In [9]:
final_df.head()

,total_name,comment,rate,source
0,카페노티드 청담,알바생 응대 최악 돈주고 불친절을 경험하네요,1점,카카오
1,카페노티드 청담,정말정말 마케팅을 잘한예.. 맛은있지만 또기다리라면 못기다림 안먹고말지 여기말고도 ...,2점,카카오
2,카페노티드 청담,먀...,3점,카카오
3,카페노티드 청담,최고예요,5점,카카오
4,카페노티드 청담,케이크 포장은 줄이 길지 않아서 오래 안 기다렸구 케이크 크기가 딱 적당해서 너무 ...,4점,카카오


In [10]:
df.to_csv("review_forModeling_강남.csv") #review_forModeling_지역이름으로 바꿔서 저장해주세요!